In [2]:
import os
import torch
import logging
from zipfile import ZipFile
from BudaOCR.Config import N_CHARSET
from huggingface_hub import snapshot_download
from BudaOCR.Modules import CRNNNetwork, OCRTrainer, WylieEncoder
from BudaOCR.Utils import shuffle_data, create_dir, build_data_paths, build_distribution_paths, read_distribution

logging.getLogger().setLevel(logging.INFO)

torch.cuda.empty_cache()
print(torch.__version__)

e:\Github\tibetan-ocr-training\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\Github\tibetan-ocr-training\.venv\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.15 (you have 1.4.14). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
kenlm python bindings are not installed. Most likely you want to install it using: pip install https://github.com/kpu/kenlm/archive/master.zip
kenlm python bindings are not installed. Most likely you want to install it using: pip install https://github.com/kpu/kenlm/archive/master.zip


2.3.1+cu118


In [3]:
charset = N_CHARSET
num_classes=len(charset)+1

In [ ]:
data_path = snapshot_download(repo_id="BDRC/Karmapa8", repo_type="dataset",  cache_dir="Datasets")

with ZipFile(f"{data_path}/data.zip", 'r') as zip:
    zip.extractall(f"{data_path}/Dataset")

dataset_path = f"{data_path}/Dataset"
image_paths, label_paths = build_data_paths(dataset_path)
image_paths, label_paths = shuffle_data(image_paths, label_paths)

print(f"Images: {len(image_paths)}, Labels: {len(label_paths)}")

output_dir = os.path.join("Output")
create_dir(output_dir)

In [5]:
dataset_path = "Datasets/Drutsa-Batch31/batch31"
image_paths, label_paths = build_data_paths(dataset_path)
image_paths, label_paths = shuffle_data(image_paths, label_paths)

print(f"Images: {len(image_paths)}, Labels: {len(label_paths)}")

Images: 5946, Labels: 5946


In [6]:
wylie_encoder = WylieEncoder(N_CHARSET)
print(wylie_encoder.num_classes())


80


In [7]:
output_dir = os.path.join(dataset_path, "Output")
create_dir(output_dir)

image_width = 3200
image_height = 100
num_classes = wylie_encoder.num_classes()

network = CRNNNetwork(image_width=image_width, image_height=image_height, num_classes=num_classes)
workers = 4

ocr_trainer = OCRTrainer(
    network=network,
    label_encoder=wylie_encoder,
    workers=workers, 
    image_width=image_width,
    image_height=image_height,
    batch_size=32, 
    output_dir=output_dir, 
    preload_labels=True
    )

ocr_trainer.init(image_paths, label_paths)

Created output directory: Datasets/Drutsa-Batch31/batch31\Output
Created output directory: Datasets/Drutsa-Batch31/batch31\Output\2024_9_20_21_54
OCR-Trainer -> Architecture: CRNN


In [ ]:
ocr_trainer.train(epochs=26, scheduler_start=20, check_cer=True)

In [10]:
data_path = "E:/Github/DataProcessing/Data/DbuMed/DrutsaB-Dataset"
distr = f"{data_path}/Output/2024_8_26_15_52/data.distribution"

train_samples, valid_samples, test_samples = read_distribution(distr)

print(f"Train Samples: {len(train_samples)}")
print(f"Val Samples: {len(valid_samples)}")
print(f"Test Samples: {len(test_samples)}")

train_images, train_labels = build_distribution_paths(data_path, train_samples)
valid_images, valid_labels = build_distribution_paths(data_path, valid_samples)
test_images, test_labels = build_distribution_paths(data_path, test_samples)

distribution = {}
distribution["train_images"] = train_images
distribution["train_labels"] = train_labels
distribution["valid_images"] = valid_images
distribution["valid_labels"] = valid_labels
distribution["test_images"] = test_images
distribution["test_labels"] = test_labels


output_dir = os.path.join(data_path, "Output")
create_dir(output_dir)

Train Samples: 3928
Val Samples: 491
Test Samples: 491


In [ ]:
image_width = 3200
network = CRNNNetwork(image_width=image_width, num_classes=num_classes)
workers = 4
ocr_trainer = OCRTrainer(network=network, workers=workers, charset=charset, image_width=3200, batch_size=32, output_dir=output_dir, preload_labels=True)

ocr_trainer.init_from_distribution(distribution)
ocr_trainer.train(epochs=48, scheduler_start=38, check_cer=True)